<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li></ul></div>

# Защита персональных данных клиентов

Нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

## Загрузка данных

Импортируем нужные библиотеки

In [1]:
# импортируем нужные библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.linear_model import LinearRegression 
import seaborn as sns
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler

Прочитаем файл /datasets/insurance.csv и сохраним его в переменную `insurance`.

In [2]:
# читаем файл и сохраняем его в insurance
insurance = pd.read_csv('/datasets/insurance.csv') 

Из описания к данным известно, что: 

* Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
* Целевой признак: количество страховых выплат клиенту за последние 5 лет.

Напишем функцию, которая выводит общую информацию о таблице, а так же посчитает пропуски и проверит на наличие дубликатов.

In [3]:
#пишем функцию summary
def summary(df):
    print(df.info())
    display(df.head())
    count=0
    for col in df.columns:
        if df.isna().sum()[col]==0:
            count+=1
        else:
            print(f'Пропущенных значений в столбце {col}: {df.isna().sum()[col]}')
    if count==len(df.columns):
        print('Пропущенных значений в таблице нет')
    print(f'Повторов в таблице {df.duplicated().sum()}')  

In [4]:
# смотрим на данные таблицы
summary(insurance)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Пропущенных значений в таблице нет
Повторов в таблице 153


Итак в таблице данные о 5000 клиентах страховой компании «Хоть потоп». Пропусков в таблице нет. Есть 153 повтора. Посмотрим на повторы.

In [5]:
# посмотрим примеры строк с полным повторениями,т.е. дублями
# отсортируем по столбцу 'Зарплата'

insurance.loc[insurance.duplicated(keep=False)].sort_values(by='Зарплата').head(10)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
2955,1,32.0,21600.0,0,0
2988,1,32.0,21600.0,0,0
361,0,50.0,24700.0,1,2
2869,0,50.0,24700.0,1,2
333,0,32.0,25600.0,1,0
4230,0,32.0,25600.0,1,0
1378,0,36.0,26400.0,0,0
2723,0,36.0,26400.0,0,0
1002,1,34.0,26900.0,0,0
1140,1,34.0,26900.0,0,0


Удалим их. Это примерно 3% от всего объема.

In [6]:
# удалим дубликаты и обновим индексацию данных
insurance = insurance.drop_duplicates().reset_index(drop=True) 

Посмотрим на взаимосвязь целевого признака и признаков.

In [7]:
# изучим взаимосвязь данных
corr = insurance.corr()\
    .sort_values('Страховые выплаты', ascending=False)\
    .reset_index()
corr[['index', 'Страховые выплаты']].style.background_gradient('coolwarm')

,index,Страховые выплаты
0,Страховые выплаты,1.000000
1,Возраст,0.654964
2,Пол,0.011565
3,Зарплата,-0.013123
4,Члены семьи,-0.039303


Целевой признак зависит от возраста клиента и не зависит от пола, зарплаты и количество членов семью застрахованного.

**Вывод**
* была открыта и изучена таблица с данными
* были удалены 153 повтора в таблице
* была найдена зависимоть целевого признака от возраста клиента

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)


Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Формула предсказаний:

$$
a = X(X^T X)^{-1} X^T y
$$

In [8]:
# разделим данные на признаки и целевой признак
X = insurance.drop(labels = "Страховые выплаты",axis = 1)
y = insurance["Страховые выплаты"]

In [9]:
# посмотрим на размерности признаков
print(X.shape,y.shape)

(4847, 4) (4847,)


In [10]:
# добавим к признакам 1-ный вектор, что соответствует w0
X = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
X.shape

(4847, 5)

Как известно, обратимой может быть только квадратная матрица. У нас в задаче нужно создать матрицу признаков - это будет матрица с 4847 строками и всего 5 столбцами. Создадим рандомно квадратную матрицу 5*5.

In [11]:
# создадим рандомно квадратную матрицу 5*5
np.random.seed(0)
R = []
for i in range(5):
    vector = np.random.rand(5)
    R.append(vector)
    
R = np.array(R)
R

array([[0.5488135 , 0.71518937, 0.60276338, 0.54488318, 0.4236548 ],
       [0.64589411, 0.43758721, 0.891773  , 0.96366276, 0.38344152],
       [0.79172504, 0.52889492, 0.56804456, 0.92559664, 0.07103606],
       [0.0871293 , 0.0202184 , 0.83261985, 0.77815675, 0.87001215],
       [0.97861834, 0.79915856, 0.46147936, 0.78052918, 0.11827443]])

Домножим матрицу признаков на обратимую матрицу.

In [12]:
# домножаем матрицу на обратимую матрицу
X_R = X.dot(R)

Обучим модель линейной регрессии и найдём метрику MSE для матрицы X и для X_R. Напишем функцию для нахождения метрики MSE.

In [13]:
# пишем функцию для нахождения метрики MSE
def metrics(X,y,name):
    model = LinearRegression()
    model.fit(X,y)
    predict = model.predict(X)
    return print("Метрика MSE для матрицы",name,"равна",mean_squared_error(y,predict))

In [14]:
# найдём значение MSE для исходной матрицы
metrics(X,y,name = 'X')

Метрика MSE для матрицы X равна 0.12527263826681603


In [15]:
# найдём значение MSE для преобразованной матрицы
metrics(X_R,y,name = 'X_R')

Метрика MSE для матрицы X_R равна 0.12527150333174444


**Ответ:** Метрика почти не изменилась

**Обоснование:** см. ниже

Предсказания:

$$
a = XPw
$$

Формула обучения:

$$
w = ((XP)^T XP)^{-1} (XP)^T y
$$

Раскроем скобки:

$$
w = (P^TX^TXP)^{-1} P^TX^T y
$$


Раскроем скобки для обратных матриц:

$$
w = P^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^T y
$$

Сокращаем произведение $(P^T)^{-1}P^T$, так как оно даёт единичную матрицу, получим:


$$
w = P^{-1}(X^TX)^{-1}X^T y
$$

Распишем предсказания:

$$
a = XPP^{-1}(X^TX)^{-1}X^T y
$$

Сокращаем произведение $P(P)^{-1}$, так как оно даёт единичную матрицу, получим:

$$
a = X(X^TX)^{-1}X^T y
$$

Получили аналогичную формулу. Значит можно сделать вывод, что домножение матрицы признаков на обратимую матрицу,никак не повлияет на значение метрик.

**Вывод**
* была найдена метрика MSE для признаков и для признаков домноженных на обратимую матрицу
* было обосновано, почему метрика MSE не изменилась

## Алгоритм преобразования

**Алгоритм**

Применим метод StandardScaler.

**Обоснование**

Суть метода: 

$$
z=(x-u)/s
$$

Где:
x - признак, который необходимо изменить
u - среднее значение столбца данного признака
s - стандратное отклонение(СКО) признака.

Данный метод приведет все признаки в диапазон 0  одно стандартное отклонение.

In [17]:
# преобразуем матрицу признаков
ss = StandardScaler()
X_ss = ss.fit_transform(X)

display(X_ss)

array([[ 0.        ,  1.0030995 ,  1.1754362 ,  0.97315092, -0.18517565],
       [ 0.        , -0.99691008,  1.76456423, -0.19011493, -0.18517565],
       [ 0.        , -0.99691008, -0.23847105, -1.89490109, -1.09546611],
       ...,
       [ 0.        , -0.99691008, -1.29890149, -0.60126924,  0.7251148 ],
       [ 0.        ,  1.0030995 , -1.06325028, -0.72160708,  1.63540526],
       [ 0.        ,  1.0030995 , -0.35629665,  0.07061707, -0.18517565]])

## Проверка алгоритма

In [18]:
# r2 исходной матрицы
model = LinearRegression()
model.fit(X,y)
predict = model.predict(X)

print("R2 исходных данных = ",r2_score(y,predict))

R2 исходных данных =  0.4302010044852067


In [19]:
# r2 обратимой матрицы
model = LinearRegression()
model.fit(X_R,y)
predict = model.predict(X_R)

print("R2 с обратимой матрицей = ",r2_score(y,predict))

R2 с обратимой матрицей =  0.4302061667047682


In [20]:
# r2 закодированной матрицы
model = LinearRegression()
model.fit(X_ss,y)
predict = model.predict(X_ss)

print("R2 закодированной матрицы = ",r2_score(y,predict))

R2 закодированной матрицы =  0.4302010044852068


**Вывод**
* метрика R2 не отличается до и после преобразований